In [1]:
import pandas as pd

#my_file = 

In [2]:
df = pd.read_csv('test.csv', encoding='latin_1', skiprows=13, sep=';')

#print(df.to_string()) 

In [3]:
new_df = df[["Buchung", "Auftraggeber/Empfänger", "Verwendungszweck", "Betrag"]]
new_df.rename(columns={
    "Buchung" : "Date",
    "Auftraggeber/Empfänger" : "Payee",
    "Verwendungszweck" : "Notes",
    "Betrag" : "Amount"
}, inplace=True)
print(new_df.to_string()) 

          Date                                               Payee                                                                                               Notes    Amount
0   11.04.2023                 PayPal Europe S.a.r.l. et Cie S.C.A  1026249471743 PP.8455.PP . Takeaway .com Payments B.V., Ihr Einkauf bei Takeaway.com Payments B.V.    -15,50
1   04.04.2023                            VISA PATREON* MEMBERSHIP                         NR XXXX 3012 INTERNET IE KAUFUMSATZ 01.04 135633 ARN24871153091088790959710     -3,57
2   04.04.2023                                                 ING                                     GIROCARD 250000901849 MONATLICHES ENTGELT GIROCARD (DEBITKARTE)     -0,99
3   04.04.2023                            AMAZON MEDIA EU S.A R.L.                                                   D01-3633538-2850237 Kindle Svcs 4Z4 A9Q7NQGX92NBX     -9,99
4   03.04.2023                                 Digitalcourage e.V.                                                 

/tmp/ipykernel_1028/1510834113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns={


In [4]:
for row in new_df.index:
    if new_df.loc[row, "Payee"].startswith('VISA'):
        new_df.at[row, "Payee"] =  new_df.loc[row, "Payee"][5:].title() # remove 'VISA ' from payee and turn UPPER to Capitalized string


In [5]:
def remove_at(i, s):
    return s[:i] + s[i+1:]

for row in new_df.index:
    if new_df.loc[row, "Payee"] == 'PayPal Europe S.a.r.l. et Cie S.C.A':
        # exported csv from ing has a space at position 35 in the notes for paypal-transactions for some reason...
        if new_df.loc[row, "Notes"][35] == ' ':
            new_df.at[row, "Notes"] = remove_at(35, new_df.at[row, "Notes"])
        # replace paypal as payee with actual payee from notes
        pos_from = new_df.loc[row, "Notes"].find(' . ') + 3
        pos_to = new_df.loc[row, "Notes"].find(',')
        actual_payee = new_df.loc[row, "Notes"][pos_from:pos_to]
        new_df.at[row, "Payee"] = actual_payee


In [6]:
s = "1025936808250 . Blizzard Entertainment SAS, Ihr Einkauf bei Blizzard E ntertainment SAS"
print(s.find(' . '))
print(s.find(','))
p = s[s.find(' . ')+3:s.find(',')]
print(p)

13
42
Blizzard Entertainment SAS


In [7]:
print(new_df.to_string()) 

          Date                                               Payee                                                                                              Notes    Amount
0   11.04.2023                          Takeaway.com Payments B.V.  1026249471743 PP.8455.PP . Takeaway.com Payments B.V., Ihr Einkauf bei Takeaway.com Payments B.V.    -15,50
1   04.04.2023                                 Patreon* Membership                        NR XXXX 3012 INTERNET IE KAUFUMSATZ 01.04 135633 ARN24871153091088790959710     -3,57
2   04.04.2023                                                 ING                                    GIROCARD 250000901849 MONATLICHES ENTGELT GIROCARD (DEBITKARTE)     -0,99
3   04.04.2023                            AMAZON MEDIA EU S.A R.L.                                                  D01-3633538-2850237 Kindle Svcs 4Z4 A9Q7NQGX92NBX     -9,99
4   03.04.2023                                 Digitalcourage e.V.                                                      

In [9]:
new_df.to_csv(path_or_buf='output.csv', sep=';', index=False)